# SOE-MIA-V1E4-002
## Modulo 2 - Data, open data y Big Data
## Práctica Nro. 3


Datos de crimen de diferentes grandes ciudades de EE.UU. que se cuentra en el DataSet "Heart Disease Prediction" del repositorio [Kaggle](https://www.kaggle.com/datasets/middlehigh/los-angeles-crime-data-from-2000).  

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# descomprimimos el archivo
# !unzip heart_2020_cleaned.csv.zip
# !unzip heart_2022_with_nans.csv.zip
# mv nMiArchivo.zip archive.zip
!unzip archive.zip

## Alternativa de descarga a al espacio de almacenamiento local de Colab

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("middlehigh/los-angeles-crime-data-from-2000")

print("Path to dataset files:", path)

100%|██████████| 502M/502M [00:12<00:00, 41.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/middlehigh/los-angeles-crime-data-from-2000/versions/3


In [ ]:
! ls -l -h $path/

total 1.9G
-rw-r--r-- 1 root root 1.7G Aug 16 01:37 'Chicage Crime Data.csv'
-rw-r--r-- 1 root root 230M Aug 16 01:37  Crime_Data_from_2020_to_Present.csv


In [ ]:
!cp $path/'Chicage Crime Data.csv' /content/Chicage_Crime_Data.csv

In [ ]:
!cp $path/'Crime_Data_from_2020_to_Present.csv' /content/Crime_Data_from_2020_to_Present.csv

In [ ]:
!ls -l -h

total 1.9G
-rw-r--r-- 1 root root 1.7G Aug 16 01:40 Chicage_Crime_Data.csv
-rw-r--r-- 1 root root 230M Aug 16 01:40 Crime_Data_from_2020_to_Present.csv
drwxr-xr-x 1 root root 4.0K Aug 14 13:36 sample_data


## Instalamos las dependencia de PySpark

In [ ]:
##
!pip install pyspark

In [ ]:
!pip install findspark

## importamos algunas librerias

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, min, max, avg, count
import findspark
findspark.init()

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Iniciar SparkSession

spark = SparkSession.builder.appName("CrimeData").getOrCreate()


## Cargamos los datos a PySpartk

In [ ]:
# Cargar datos CSV
data = spark.read.csv("/content/Chicage_Crime_Data.csv", header=True)


In [ ]:
data2 = spark.read.csv("/content/Crime_Data_from_2020_to_Present.csv", header=True)

In [ ]:
data.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [ ]:
len(data.columns)

22

In [ ]:
data2.columns

In [ ]:
print(len(data2.columns))

28


In [ ]:
# Imprimimos el Schema
data.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



## Exploración de datos

In [ ]:
# revisamos la cantidad de registro de nuestro dataset
row_number = data.count()
print('la cantidad de files es',row_number)

la cantidad de files es 7391187


In [ ]:
data.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [ ]:
data.head(5)

[Row(ID='5741943', Case Number='HN549294', Date='08/25/2007 09:22:18 AM', Block='074XX N ROGERS AVE', IUCR='0560', Primary Type='ASSAULT', Description='SIMPLE', Location Description='OTHER', Arrest='false', Domestic='false', Beat='2422', District='024', Ward='49', Community Area='1', FBI Code='08A', X Coordinate=None, Y Coordinate=None, Year='2007', Updated On='08/17/2015 03:03:40 PM', Latitude=None, Longitude=None, Location=None),
 Row(ID='25953', Case Number='JE240540', Date='05/24/2021 03:06:00 PM', Block='020XX N LARAMIE AVE', IUCR='0110', Primary Type='HOMICIDE', Description='FIRST DEGREE MURDER', Location Description='STREET', Arrest='true', Domestic='false', Beat='2515', District='025', Ward='36', Community Area='19', FBI Code='01A', X Coordinate='1141387', Y Coordinate='1913179', Year='2021', Updated On='11/18/2023 03:39:49 PM', Latitude='41.917838056', Longitude='-87.755968972', Location='(41.917838056, -87.755968972)'),
 Row(ID='26038', Case Number='JE279849', Date='06/26/202

In [ ]:
## Nos creamos una vista
data.createOrReplaceTempView("Crime") #temporary view


In [ ]:

Years = spark.sql('SELECT Year, count(*) from Crime group by Year order by count(1) asc')
# Years.printSchema()
Years.show()


+----+--------+
|Year|count(1)|
+----+--------+
|NULL|       1|
|2021|    2062|
|2022|    4279|
|2023|   89093|
|2024|   92598|
|2020|  152831|
|2019|  260487|
|2015|  264536|
|2018|  268437|
|2017|  268804|
|2016|  269573|
|2014|  275649|
|2013|  307396|
|2012|  336212|
|2011|  351925|
|2010|  370442|
|2009|  392796|
|2008|  427135|
|2007|  437050|
|2006|  448149|
+----+--------+
only showing top 20 rows



In [ ]:
data2.createOrReplaceTempView("Crime2020") #temporary view


In [ ]:
data2.columns

['DR_NO',
 'Date Rptd',
 'DATE OCC',
 'TIME OCC',
 'AREA',
 'AREA NAME',
 'Rpt Dist No',
 'Part 1-2',
 'Crm Cd',
 'Crm Cd Desc',
 'Mocodes',
 'Vict Age',
 'Vict Sex',
 'Vict Descent',
 'Premis Cd',
 'Premis Desc',
 'Weapon Used Cd',
 'Weapon Desc',
 'Status',
 'Status Desc',
 'Crm Cd 1',
 'Crm Cd 2',
 'Crm Cd 3',
 'Crm Cd 4',
 'LOCATION',
 'Cross Street',
 'LAT',
 'LON']

In [ ]:
Years2 = spark.sql('SELECT Year, count(*) from Crime2020 group by Year order by count(1) asc')
# Years.printSchema()
Years2.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Year` cannot be resolved. Did you mean one of the following? [`LAT`, `LON`, `AREA`, `DR_NO`, `Status`].; line 1 pos 7;
'Sort [count(1) ASC NULLS FIRST], true
+- 'Aggregate ['Year], ['Year, count(1) AS count(1)#220L]
   +- SubqueryAlias crime2020
      +- View (`Crime2020`, [DR_NO#78,Date Rptd#79,DATE OCC#80,TIME OCC#81,AREA#82,AREA NAME#83,Rpt Dist No#84,Part 1-2#85,Crm Cd#86,Crm Cd Desc#87,Mocodes#88,Vict Age#89,Vict Sex#90,Vict Descent#91,Premis Cd#92,Premis Desc#93,Weapon Used Cd#94,Weapon Desc#95,Status#96,Status Desc#97,Crm Cd 1#98,Crm Cd 2#99,Crm Cd 3#100,Crm Cd 4#101,LOCATION#102,Cross Street#103,LAT#104,LON#105])
         +- Relation [DR_NO#78,Date Rptd#79,DATE OCC#80,TIME OCC#81,AREA#82,AREA NAME#83,Rpt Dist No#84,Part 1-2#85,Crm Cd#86,Crm Cd Desc#87,Mocodes#88,Vict Age#89,Vict Sex#90,Vict Descent#91,Premis Cd#92,Premis Desc#93,Weapon Used Cd#94,Weapon Desc#95,Status#96,Status Desc

In [ ]:
df = Years.toPandas()

In [ ]:
df

,Year,count(1)
0,None,1
1,2021,2062
2,2022,4279
3,2023,89093
4,2024,92598
5,2020,152831
6,2019,260487
7,2015,264536
8,2018,268437
9,2017,268804


In [ ]:
import plotly.express as px

In [ ]:
# Realizamos un gráfico on Plotly
dfP = df.sort_values(by='Year',ascending=True)
fig = px.histogram(df, x="Year", y="count(1)")
fig.update_layout(bargap=0.2,title='Histórico de Crimen en Chicago')
fig.show()


In [ ]:
df.dtypes

,0
Year,object
count(1),int64


## Graficamos un resultado

In [ ]:
# Visualizamos un resultado
dfP['Year_fix'] = pd.to_numeric(df['Year'], errors='coerce')
# dfP = df.sort_values(by='Year',ascending=True)
fig = px.histogram(dfP, x="Year_fix", y="count(1)",nbins=30)
fig.update_layout(bargap=0.2,title='Histórico de Crimen en Chicago')
fig.show()

utilizamos las funciones de los dataframe

In [ ]:

# probamos la conversión de Dataframe PySpark a Pandas
from pyspark.sql import functions as F
Asaltos = data.groupBy('Primary Type').agg(F.count('*'))
# Asaltos.printSchema()
# Asaltos.show()


In [ ]:
AsaltosDF = Asaltos.toPandas()


In [ ]:
# Generamos una gráfica en plotly
AsaltosDF.columns = ['Asaltos','Nro']
fig = px.histogram(AsaltosDF, x="Asaltos", y="Nro")
fig.update_layout(bargap=0.2,title='Histórico de Crimen en Chicago')
fig.show()

In [ ]:
Asaltos2002 = data['Year','Primary Type'][data['Year']=='2002']
Asaltos2002.show()

+----+--------------------+
|Year|        Primary Type|
+----+--------------------+
|2002|               THEFT|
|2002|               THEFT|
|2002|           NARCOTICS|
|2002|             ASSAULT|
|2002|  DECEPTIVE PRACTICE|
|2002|               THEFT|
|2002|           NARCOTICS|
|2002|           NARCOTICS|
|2002|               THEFT|
|2002|OFFENSE INVOLVING...|
|2002|OFFENSE INVOLVING...|
|2002|               THEFT|
|2002|CRIMINAL SEXUAL A...|
|2002|               THEFT|
|2002|             ROBBERY|
|2002|   CRIMINAL TRESPASS|
|2002|             BATTERY|
|2002|     CRIMINAL DAMAGE|
|2002|             BATTERY|
|2002|               THEFT|
+----+--------------------+
only showing top 20 rows



In [ ]:
AsaltosDF = Asaltos2002.groupBy('Primary Type').agg(F.count('*'))
AsaltosDF = AsaltosDF.toPandas().sort_values(by='count(1)',ascending=False)

In [ ]:
AsaltosDF.shape

(30, 2)

In [ ]:
AsaltosDF

,Primary Type,count(1)
6,THEFT,98331
7,BATTERY,94150
14,CRIMINAL DAMAGE,55941
15,NARCOTICS,51789
16,OTHER OFFENSE,32600
4,ASSAULT,31521
17,BURGLARY,25623
5,MOTOR VEHICLE THEFT,25121
8,ROBBERY,18522
3,CRIMINAL TRESPASS,13881


In [ ]:
# gemerar un grafico de histograma con la variable df con plotly
AsaltosDF.columns = ['Year','Asaltos']
fig = px.histogram(AsaltosDF, x="Year", y="Asaltos")
fig.update_layout(bargap=0.2,title='Histórico de Crimen en Chicago en el año 2002')
fig.show()